In [ ]:
# Review Reply Conditional Workflow
# This notebook demonstrates conditional branching in LangGraph for automated review response generation
# The workflow detects sentiment, diagnoses issues, and generates appropriate replies

# Import required modules
from langchain_groq import ChatGroq                    # Groq LLM integration
from langgraph.graph import StateGraph, START, END    # LangGraph workflow components
from pydantic import BaseModel, Field                 # For structured output schema
from typing import Literal, TypedDict                 # Type-safe state definition
from dotenv import load_dotenv                        # Environment variable management

# Load environment variables (API keys)
load_dotenv(dotenv_path='../.env')

True

In [ ]:
# Initialize the Large Language Model
model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
# Define Sentiment Output Schema
# Used for structured LLM output on sentiment detection
class SentimentSchema(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Sentiment of the review')

In [ ]:
# Define Diagnosis Output Schema
# Used for structured LLM output on issue diagnosis
class DiagnosisSchema(BaseModel):
    issue_type : Literal['UX', 'Performance', 'Bug', 'Support', 'Other'] = Field(description='The category of issue mentioned in the review')
    tone : Literal['angry', 'frustrated', 'disappointed', 'calm'] = Field(description='The emotional tone expressed by the user')
    urgency : Literal['low', 'medium','high'] = Field(description='How urgent or critical the issue appears to be')

In [ ]:
# Configure the LLM for structured output
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [ ]:
# Example: Sentiment Detection Prompt
prompt = 'What is the sentiment of the following review : The phone is super bad. It is outright trash. Total waste of money.'

structured_model.invoke(prompt).sentiment

'negative'

In [ ]:
# Define the Review State Schema
# Tracks review text, sentiment, diagnosis, and generated response
class ReviewState(TypedDict):
    review : str
    sentiment : Literal['positive', 'negative']
    diagnosis : dict
    response : str

In [ ]:
def find_sentiment(state: ReviewState): 
    """
    Step 1: Detect sentiment of the review using LLM
    Updates state with detected sentiment
    """
    prompt = f'For the following review find out the sentiment \n {state["review"]}'
    sentiment = structured_model.invoke(prompt).sentiment
    state['sentiment'] = sentiment
    return state

def check_sentiment(state: ReviewState) -> Literal['positive_response', 'run_diagnosis']:
    """
    Step 2: Conditional branching based on sentiment
    Returns next node name
    """
    if state["sentiment"] == 'positive':
        return 'positive_response'
    else :
        return 'run_diagnosis'

def positive_response(state: ReviewState):
    """
    Step 3a: Generate a warm thank you message for positive reviews
    Updates state with response
    """
    prompt = f'Write a warm thank you message in reponse to this review\n\n {state["review"]} \n Also , kindly ask the user to leave a feedback on our website'
    response = model.invoke(prompt).content
    state['response'] = response
    return state

def run_diagnosis(state: ReviewState):
    """
    Step 3b: Diagnose negative review for issue type, tone, and urgency
    Updates state with diagnosis
    """
    prompt = f"Diagnose this negative review: \n\n {state['review']}\n Return issue_type, tone and urgency."
    response = structured_model2.invoke(prompt)
    state['diagnosis'] = response.model_dump()
    return state

def negative_response(state: ReviewState):
    """
    Step 3c: Generate empathetic resolution message for negative reviews
    Updates state with response
    """
    diagnosis = state["diagnosis"]
    prompt = f"You are a support assistant. The user has a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}' , and marked urgency as '{diagnosis['urgency']}'. Write an empathetic, helpful resolution message"
    response = model.invoke(prompt).content
    state['response'] = response
    return state

In [ ]:
# Build the Conditional Review Reply Workflow Graph
# The workflow branches based on sentiment and generates appropriate responses

graph = StateGraph(ReviewState)

# Add nodes for each step
graph.add_node('find_sentiment', find_sentiment)              # Detect sentiment
graph.add_node('positive_response', positive_response)        # Generate thank you for positive reviews
graph.add_node('run_diagnosis', run_diagnosis)                # Diagnose negative reviews
graph.add_node('negative_response', negative_response)        # Generate resolution for negative reviews

# Define edges for sequential and conditional flow
graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_sentiment)
graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

# Compile the graph into an executable workflow
workflow = graph.compile()

In [ ]:
# Visualize the Workflow Structure
# Shows conditional branching for review responses
workflow.get_graph().print_ascii()

                    +-----------+                      
                    | __start__ |                      
                    +-----------+                      
                          *                            
                          *                            
                          *                            
                  +----------------+                   
                  | find_sentiment |                   
                  +----------------+                   
                  ..             ...                   
               ...                  ...                
             ..                        ..              
  +---------------+                      ..            
  | run_diagnosis |                       .            
  +---------------+                       .            
          *                               .            
          *                               .            
          *                               .     

In [ ]:
# Execute the Workflow with Sample Review
# Test the review reply workflow for a sample negative review
initial_state = {
    'review' : 'Food was cold when served, portions were tiny, and the staff seemed disinterested. Not worth the price.'
}

workflow.invoke(initial_state)

{'review': 'Food was cold when served, portions were tiny, and the staff seemed disinterested. Not worth the price.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Performance',
  'tone': 'angry',
  'urgency': 'medium'},
 'response': "I can sense your frustration with the performance issue you're experiencing, and I'm truly sorry that it's causing you distress. I'm here to help you resolve this problem as efficiently as possible.\n\nFirstly, please know that I'm committed to providing you with a solution that meets your needs. I understand that this issue is important to you, and I'll do my best to address it with the urgency it deserves.\n\nTo better assist you, could you please provide more details about the performance issue you're encountering? This will enable me to gain a deeper understanding of the problem and work towards a more effective solution.\n\nIn the meantime, I'd like to offer some potential troubleshooting steps that may help alleviate the issue. However, if